## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-20-01-10-38 +0000,nyt,Suspected Killer of M.I.T. Professor Studied W...,https://www.nytimes.com/2025/12/19/us/mit-prof...
1,2025-12-20-01-07-44 +0000,bbc,US carries out 'massive' strike against IS in ...,https://www.bbc.com/news/articles/c5yq7zzw618o...
2,2025-12-20-01-04-26 +0000,wapo,Saudi Arabia’s great game,https://www.washingtonpost.com/world/2025/12/1...
3,2025-12-20-01-04-00 +0000,wsj,Five Days of Mystery: How the Brown University...,https://www.wsj.com/us-news/law/five-days-of-m...
4,2025-12-20-01-00-08 +0000,nyt,How a Reddit Post Helped the Police Identify a...,https://www.nytimes.com/2025/12/19/us/brown-mi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
118,trump,35
85,epstein,22
39,new,19
352,ukraine,17
22,brown,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2025-12-19-21-06-58 +0000,nypost,DOJ releases thousands of Jeffrey Epstein case...,https://nypost.com/2025/12/19/us-news/doj-rele...,106
29,2025-12-19-23-51-09 +0000,nyt,Trump Is Rarely Mentioned in New Epstein Files,https://www.nytimes.com/2025/12/19/us/trump-ep...,93
121,2025-12-19-20-31-32 +0000,nypost,Trump says he will haul in ‘rich’ health insur...,https://nypost.com/2025/12/19/us-news/trump-sa...,82
322,2025-12-19-05-54-00 +0000,wsj,Brown and MIT Shooting Suspect Found Dead in N...,https://www.wsj.com/us-news/brown-shooting-sus...,81
289,2025-12-19-10-04-01 +0000,bbc,Brown University and MIT professor shooting su...,https://www.bbc.com/news/articles/cx2e14z2j9jo...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,106,2025-12-19-21-06-58 +0000,nypost,DOJ releases thousands of Jeffrey Epstein case...,https://nypost.com/2025/12/19/us-news/doj-rele...
322,81,2025-12-19-05-54-00 +0000,wsj,Brown and MIT Shooting Suspect Found Dead in N...,https://www.wsj.com/us-news/brown-shooting-sus...
55,70,2025-12-19-22-56-48 +0000,nypost,Rubio warns Ukraine war won’t end in surrender...,https://nypost.com/2025/12/19/world-news/rubio...
310,52,2025-12-19-08-34-47 +0000,nyt,Australia Announces Plan to Buy Back Guns in W...,https://www.nytimes.com/2025/12/19/world/asia/...
34,51,2025-12-19-23-43-00 +0000,wsj,"Rep. Elise Stefanik (R., N.Y.) said she would ...",https://www.wsj.com/politics/elections/elise-s...
74,51,2025-12-19-22-21-47 +0000,nypost,Elon Musk wins appeal to restore $56B Tesla pa...,https://nypost.com/2025/12/19/business/elon-mu...
121,47,2025-12-19-20-31-32 +0000,nypost,Trump says he will haul in ‘rich’ health insur...,https://nypost.com/2025/12/19/us-news/trump-sa...
76,42,2025-12-19-22-17-50 +0000,missionlocal,Nightmare before Christmas: Fillmore low-incom...,https://missionlocal.org/2025/12/sf-thomas-pai...
312,41,2025-12-19-08-13-02 +0000,nypost,EU leaders agree on 90 billion euro loan to Uk...,https://nypost.com/2025/12/19/world-news/eu-le...
326,37,2025-12-19-05-12-07 +0000,nypost,Wisconsin Judge Hannah Dugan guilty of obstruc...,https://nypost.com/2025/12/19/us-news/judge-ha...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
